In [1]:
import requests
import pandas as pd

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests


from tqdm import tqdm

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

In [2]:
lista_pelis = []


for año in range(1990, 1993):
    
    sleep(10)
    

    url = "https://moviesdatabase.p.rapidapi.com/titles"

    querystring = {"genre":"Drama","year":"1990","page":"1", "limit": "50"}

    headers = {
        "X-RapidAPI-Key": "b1101936b5mshb3f446f3203eb54p1c27c3jsna214ad23b3e2",
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
    print(response.json())
    print(response.status_code)
    # con esto entramos en la key donde encontraremos las pelis, por cada pagina tenemos solo 10 resultados
    pelis = response.json()["results"]
    print(pelis)

    
    for indice, peli in enumerate(pelis):
        resultados = []
        resultados.append(pelis[indice]["titleText"]["text"])
        resultados.append(pelis[indice]["titleType"]["text"])
        resultados.append(pelis[indice]["releaseYear"]["year"])
    
        resultados.append(pelis[indice]["id"])
        
        # hay algunas pelis/series que no tienen mes de estreno, por lo que en esos casos lo ponemos como desconocido
        try:
            resultados.append(pelis[indice]["releaseDate"]["month"])
        except:    
            resultados.append("Desconocido")
            
        lista_pelis.append(tuple(resultados))

{'page': '1', 'next': '/titles?page=2&limit=50&year=1990&genre=Drama', 'entries': 50, 'results': [{'_id': '61e580c3971a4a15d641458b', 'id': 'tt0059325', 'primaryImage': {'id': 'rm3431504896', 'width': 292, 'height': 475, 'url': 'https://m.media-amazon.com/images/M/MV5BMTc0Mzk0Mjg2MF5BMl5BanBnXkFtZTcwMzU4NzAyMQ@@._V1_.jpg', 'caption': {'plainText': 'Jahrgang 45 (1990)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Jahrgang 45', '__typename': 'TitleText'}, 'originalTitleText': {'text': 'Jahrgang 45', '__typename': 'TitleText'}, 'releaseYear': {'year': 1990, 'endYear': None, '__typename': 'YearRange'}, 'releaseDate': {'day': 11, 'month': 10, 'year': 1990, '__typename': 'ReleaseDate'}}, {'_id': '61e580c8d8f3c0931e414665', 'id': 'tt0059900', 'primaryImage': {'id': 'rm2847710208', 'width': 290, 'height': 475, 'url': 'https://m.media-amazon.com/images/M

In [6]:
lista_pelis[:3]

[('Jahrgang 45', 'Movie', 1990, 'tt0059325', 10),
 ('Wenn du groß bist, lieber Adam', 'Movie', 1990, 'tt0059900', 10),
 ('Vojtech, receny sirotek', 'Movie', 1990, 'tt0065188', 10)]

In [17]:
lista_peli_cortas = lista_pelis[7:10]
lista_peli_cortas

[('Shattered Dreams', 'TV Movie', 1990, 'tt0090008', 4),
 ('Asfour Stah', 'Movie', 1990, 'tt0090665', 3),
 ('Nonstop', 'Short', 1990, 'tt0093646', 2)]

# Actores por peli

In [20]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

lista_tupla_actores = []
sleep(3)
for tupla in tqdm(lista_peli_cortas):
    driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)
    sleep(3)
    driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-66ec1b32-0.bGqWwh > div > div:nth-child(2) > ul > li:nth-child(1) > a").click()
    sleep(3)
    
    lista_actores = []
    for i in range(2, 12):
        try:
            lista_actores.append(driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a").text)
           
        except:            
            break
    lista_actores.append(tupla[-1])
    lista_tupla_actores.append(tuple(lista_actores))

    driver.back()

driver.close()   


100%|██████████| 3/3 [00:52<00:00, 17.54s/it]


In [21]:
lista_tupla_actores

[('Lindsay Wagner',
  'Michael Nouri',
  'Georgann Johnson',
  'James Karen',
  'Ken Jenkins',
  'Irene Miracle',
  'Stan Ivar',
  'Bryan Clark',
  'Tom Dahlgren',
  'Jay R. Ferguson',
  'tt0090008'),
 ('Selim Boughedir',
  'Mustapha Adouani',
  'Rabiah Ben-Abdullah',
  'Mohamed Driss',
  'Hélène Catzaras',
  'Fatma Ben Saïdane',
  'Abdelhamid Guayas',
  'Jamel Sassi',
  'Radhouane Meddeb',
  'Carolyn Chelby',
  'tt0090665'),
 ('Esko Nikkari',
  'Armi Sillanpää',
  'Carl-Kristian Rundman',
  'Jane Sarlund',
  'Mikko Rasila',
  'Niko Saarela',
  'Vesa Vierikko',
  'Pertti Sveholm',
  'Elina Salo',
  'Esko Salervo',
  'tt0093646')]

In [ ]:
#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.dpBDvu > a

In [ ]:
#fullcredits_content > table.cast_list > tbody > tr:nth-child(2) > td:nth-child(2)

In [ ]:
lista_tupla_actores

[('Monika Hildebrand',
  'Rolf Römer',
  'Paul Eichbaum',
  'Holger Mahlich',
  'Gesine Rosenberg',
  'Walter Stolp',
  'Werner Kanitz',
  'Ingo Koster',
  'Anita Okon',
  'Ruth Kommerell',
  'tt0059325'),
 ('Stephan Jahnke',
  'Gerry Wolff',
  'Manfred Krug',
  'Daisy Granados',
  'Rolf Römer',
  'Hanns Anselm Perten',
  'Wolfgang Greese',
  'Günther Simon',
  'Mathilde Danegger',
  'Fred Delmare',
  'tt0059900'),
 ('Petr Forman',
  'Barbora Lukesová',
  'Jana Riháková',
  'Vlastimil Zavrel',
  'Jirí Hájek',
  'Bretislav Rychlík',
  'Jaroslav Mares',
  'Miroslav Zelenka',
  'Petr Popelka',
  'Eva Holubová',
  'tt0065188'),
 ('Jason Ledger',
  'Bonnie Sher',
  'Kyoko Masu',
  'Hiroshi Murakawa',
  'Miyuki Yama',
  'Yoshihiro Sakamoto',
  'Clown Kumagoro',
  'Clown Ton',
  'Squeeky the Clown',
  'Okinaya Trio',
  'tt0068494'),
 ('Michael Petrovitch',
  'Philip Latham',
  'Don Fellows',
  'Michael Gwynn',
  'Nicholas Parsons',
  'Toby Robins',
  'Tessa Wyatt',
  'Derren Nesbitt',
  'Nige

# Caracteristicas de los actores

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()

driver.get("https://www.imdb.com/")
lista_detalles_actores = []

for actor in tqdm(lista_tupla_actores[0][: 3]):
    lista_actores = []
    print(actor)
    sleep(7)
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
    
    sleep(7)
    driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-383f2ac5-0.bfcGjo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1)").click()
    try: 
        año = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.femeOx > div > div.sc-e226b0e3-11.gOzomZ > section > aside > div > span:nth-child(2)").text.split(",")[-1] 
    except:
        año = "desconocido"
    conocido = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.jGufEe > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid").text
    que_hace = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div > ul > li").text
    lista_caracteristicas = []
    
    lista_caracteristicas.append(año)
    lista_caracteristicas.append(conocido)
    lista_caracteristicas.append(que_hace)
    lista_caracteristicas.append(actor)
    lista_detalles_actores.append(tuple(lista_caracteristicas))
    driver.back()
    

driver.close()



  0%|          | 0/3 [00:00<?, ?it/s]

Monika Hildebrand


 33%|███▎      | 1/3 [00:26<00:52, 26.44s/it]

Rolf Römer


 67%|██████▋   | 2/3 [00:46<00:22, 22.79s/it]

Paul Eichbaum


100%|██████████| 3/3 [01:07<00:00, 22.34s/it]


In [ ]:
lista_detalles_actores

[(' 1941',
  "Born in '45\n6.4\nLisa, genannt Li\n1990\nDie Glatzkopfbande\n5.5\nWaltraud\n1963\nJetzt und in der Stunde meines Todes\n7.4\nAngelika Hecht\n1963\nDas verhexte Fischerdorf\n5.1\nEvelyn\n1962",
  'Actress',
  'Monika Hildebrand'),
 (' 1935',
  'He Du!\n6.9\nMaxe Krahl(as Rolf Specht)\n1970\nTecumseh\n5.6\nSimon McKew\n1972\nChingachgook, die grosse Schlange\n6.0\nWildtöterDeerslayer\n1967\nDer Mord, der nie verjährt\nLiepmann\n1968',
  'Actor',
  'Rolf Römer'),
 ('desconocido', "Born in '45\n6.4\nMogul\n1990", 'Actor', 'Paul Eichbaum')]